# TensorFlow Data Augmentation 

# <u>Authors:</u>
## 1. Matthias Bartolo ID: 0436103L
## 2. Luke Cardona ID: 0011803H
## 3. Jerome Agius ID: 0353803L
## 4. Isaac Muscat ID: 0265203L

## <u>Installed Packages</u>

In [1]:
# !pip install tensorflow 
# !pip install --upgrade tensorflow

## <u>Packages</u>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os 
import tensorflow as tf
%matplotlib inline
from IPython.display import Image
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomCrop
from scipy.ndimage import gaussian_filter

## <u>Loading required images</u>

1. Files should be saved into Normal Images
2. Emoji Images should be saved for every 50 or so images.
3. Train Classifier

In [3]:
#Create dictionary for images
original_imgs = []

# Iterate through each image in the folder
for mainfolders in [filename for filename in os.listdir("COTSDataset/") if filename.startswith("Part")]:
    for folders in os.listdir("COTSDataset/" + mainfolders):
        for image in os.listdir("COTSDataset/" + mainfolders + "/" + folders):
            if (image.endswith("colour.jpeg")):
                #Save image in dictionary
                tempImage = cv2.imread("COTSDataset/" + mainfolders + "/" + folders + "/" + image)
                # Convert the image to RGB format (cv2 reads in BGR)
                tempImage = cv2.cvtColor(tempImage, cv2.COLOR_BGR2RGB)
                
                # Adding the image to the list of original images
                original_imgs.append(tempImage)
                cv2.imwrite(os.path.join("Test Images/NormalImages/", image), cv2.cvtColor(tempImage, cv2.COLOR_RGB2BGR))

## <u>Data Augmentation Functions</u> 

**AlterImage** - This function executes the passed data augmentation function on the passed image. The possible functions include:  

- Rotation()
- Brightness()
- HorizontalFlip()
- VerticalFlip()
- Shear()
- Translation()

In [5]:
def AlterImage(img, function):
    #Convert the image to a numpy array
    img_array = tf.keras.preprocessing.image.img_to_array(img)

    #Expand the dimensions of the image array to (1, height, width, channels)
    img_array = tf.expand_dims(img_array, 0)

    #Create an iterator for the data generator
    it = function[0].flow(img_array)

    #Get the first augmented image from the iterator
    augmented_img_array = it.next()[0]

    #Convert the augmented image array back to an image
    augmented_img = tf.keras.preprocessing.image.array_to_img(augmented_img_array)
    
    return augmented_img, function[1]

**Rotation** - This function is used in conjunction with the AlterImage function to rotate the image randomly within a set range. 

In [6]:
def Rotation(RotationValue):
    datagen = ImageDataGenerator(rotation_range=RotationValue)
    return datagen, "Rotation Image"

**Brightness** - This function is used in conjunction with the AlterImage function to adjusts the brightness of the image within a set range.

In [7]:
def Brightness(BrightnessValue):
    datagen = ImageDataGenerator(
        # Randomly adjust the brightness of the images by up to 30%
        brightness_range=(BrightnessValue,BrightnessValue),
    )
    return datagen, "Brightness Image"

**Contrast** - This function adjusts the contrast of the passed image in between a certain range.

In [8]:
def Contrast(img, ContrastValue):
    contrast_img = tf.image.random_contrast(img, 0, ContrastValue)
    return contrast_img, "Contrast Image"

**Saturation** - This function adjusts the saturation of the passed image between a certain range.

In [9]:
def Saturation(img, SaturationValue):
    saturation_img = tf.image.adjust_saturation(img, SaturationValue)
    return saturation_img, "Saturation Image"

**Hue** - This function adjusts the hue of the passed image between a certain range.

In [10]:
def Hue(img, HueValue):
    hue_img = tf.image.random_hue(img, HueValue)
    return hue_img, "Hue Image"

**CenterCrop** - This function crops the center of the passed image and returns the cropped out section.

In [11]:
def CenterCrop(img):
    height, width = tf.shape(img)[0], tf.shape(img)[1]

    # Calculate the crop offset for the height and width dimensions
    offset_height = tf.cast((height - 400) / 2, tf.int32)
    offset_width = tf.cast((width - 400) / 2, tf.int32)

    # Crop the image using the calculated offsets and crop size
    centre_crop_img = tf.image.crop_to_bounding_box(img, offset_height, offset_width,400, 400)
    return centre_crop_img, "Center Crop Image"

**HorizontalFlip** - This function is used in conjunction with the AlterImage function to flip the image horizontally in relation to its center point.

In [12]:
def HorizontalFlip(img):
    horizontal_flip_img = tf.image.flip_left_right(img)
    return horizontal_flip_img, "Horizontal Flip Image"

**VerticalFlip** - This function is used in conjunction with the AlterImage function to flip the image vertically in relation to its center point.

In [13]:
def VerticalFlip(img):
    vertical_flip_img = tf.image.flip_up_down(img)
    return vertical_flip_img,"Vertical Flip Image"

**Shear** - This function is used in conjunction with the AlterImage function to shear the image wihtin a set range.

In [14]:
def Shear(ShearValue):
    # Create an instance of ImageDataGenerator with the desired transformations
    datagen = ImageDataGenerator(
        # Apply shear transformation to the images
        shear_range=ShearValue,
    )
    return datagen, "Shear Image"

**Gamma** - This function alters the gamma values of the passed image within a set range.

In [15]:
def Gamma(img, GammaValue):
    # Apply Gamma Transformation according to the GammaValue
    gamma_img = tf.image.adjust_gamma(img, GammaValue)
    return gamma_img, "Gamma Image"

**Translation** - This function is used in conjunction with the AlterImage function to translate the entire image along both axis within a set range. 

In [16]:
def Translation(TranslationValue):
    datagen = ImageDataGenerator(
    # Randomly translate the images horizontally according to the Translation Value
    width_shift_range=TranslationValue/10,
    # Randomly translate the images vertically according to the Translation Value
    height_shift_range=TranslationValue/10,
    # Fill in any empty pixels with the nearest pixel value
    fill_mode='nearest')
    return datagen, "Translation Image"

**GaussianBlur** - This function applies the GaussianBlur kernel onto the passed image

In [17]:
def GaussianBlur(img, GBlurValue):
    # Apply Gaussian blur
    gaussian_blur_img = gaussian_filter(img, sigma=GBlurValue)
    # Convert back to image
    gaussian_blur_img = tf.keras.preprocessing.image.array_to_img(gaussian_blur_img)
    return gaussian_blur_img, "Gaussian Blur Image"

**AugmentImg** - This function is used to apply all the image augmentation functions discussed above on the passed image and return a list of augemented images. 

In [18]:
def AugmentImg(img):
    RotationValue = 30
    BrightnessValue = 0.6
    ContrastValue = 1
    SaturationValue = 1
    ShearValue = 30
    HueValue = 0.1
    GammaValue = 1
    GBlurValue = 31
    TranslationValue = 0.5
    # Augmented_Img = {"Rotations":[], "Brightness":[],"Contrast":[],"Saturation":[],
    #                  "Hue":[],"Flipped":[],"Shearing":[],
    #                 "CenterCrop":[],"Gamma":[],"GaussianBlur":[],"Translation": []}
    Augmented_Img = {"Rotations":[], "Shearing":[],
                    "CenterCrop":[],"Translation": []}
    #Executing all augmentation function fives times per image and storing the results
    for x in range(1):

        
        #Photometric Augmentation
        # Augmented_Img["Brightness"].append(AlterImage(img,Brightness(BrightnessValue)))
        # Augmented_Img["Contrast"].append(Contrast(img,ContrastValue))
        # Augmented_Img["Saturation"].append(Saturation(img,SaturationValue))
        # Augmented_Img["Hue"].append(Hue(img,HueValue))
        # Augmented_Img["Gamma"].append(Gamma(img,GammaValue))
        # Augmented_Img["GaussianBlur"].append(GaussianBlur(img,GBlurValue))
        
        #Geometric Augmentation
        Augmented_Img["Rotations"].append(AlterImage(img, Rotation(RotationValue)))
        Augmented_Img["CenterCrop"].append(CenterCrop(img))
        # Augmented_Img["Flipped"].append(HorizontalFlip(img))
        # Augmented_Img["Flipped"].append(VerticalFlip(img))
        Augmented_Img["Shearing"].append(AlterImage(img,Shear(ShearValue)))
        Augmented_Img["Translation"].append(AlterImage(img,Translation(TranslationValue)))
        
        #Updating the parameter values
        RotationValue += 30
        BrightnessValue += 0.6
        ContrastValue += 1
        SaturationValue += 1
        HueValue += 0.05
        ShearValue += 30
        GammaValue += 1
        GBlurValue += 30
        TranslationValue += 0.5
    return Augmented_Img

In [19]:
AugmentedImgs = []

for img in original_imgs: 
    AugmentedImgs.append(AugmentImg(img))

# <u>Output Display</u>

This method displays all the altered images in an orderly manner for presentation purposes.

In [20]:
# rows, cols = 11,5

# #Looping through all the augmented images
# for imgSet in AugmentedImgs:
#     fig, ax = plt.subplots(11, 5, figsize=(10,20))
#     row = 0
#     for key in imgSet:
#         for cnt in range(5):
#             col = cnt % 5
            
#             #Setting the image
#             ax[row][col].imshow(imgSet[key][cnt][0])
            
#             #Setting the title
#             ax[row][col].set_title(imgSet[key][cnt][1]+" "+str(cnt+1)) 
            
#             #Removing the axes
#             ax[row][col].axis('off') 
#             fig.tight_layout()
#         row += 1 

**SaveImages** - This function saves the passed list of images to the specified directory.

In [21]:
def SaveImages(listOfImages, directory):
    if not os.path.exists(directory):
       # Create a new directory if it does not exist
       os.makedirs(directory)
    
    count = 1
    for imgSet in listOfImages:
        
        for key in imgSet:
            #Saving the images
            tf.keras.utils.save_img(directory + "/Item_"+str(count)+"_"+imgSet[key][0][1]+"_"+str(1)+".jpeg",imgSet[key][0][0], None, 'JPEG', True)
        count += 1

In [22]:
SaveImages(AugmentedImgs, "./Test Images/NormalImages")